# Настройка среды

In [1]:
import os
import sys
sys.path.append(os.path.abspath("../.."))

In [7]:
from belashovplot import TiledPlot
from utilities import *
from utilities.filters import Gaussian, Window
from elements.abstracts import AbstractModulator, AbstractPropagator
from elements.modulators import Lens, PhaseModulator, AmplitudeModulator
from elements.propagators import FurrierPropagation, ConvolutionalPropagation
from elements.composition import CompositeModel, HybridModel
from elements.wrappers import CudaMemoryChunker, Incoherent
from elements.detectors import ClassificationDetectors, MatrixDetectors
from elements.simple import AdjustSize
from parameters import FigureWidthHeight, FontLibrary
from tqdm import tqdm
from math import sin, sqrt
import torch
import numpy
import pandas
from copy import deepcopy
from itertools import product
from typing import Iterable
from cluster import train, confusion, execute, epochs

In [3]:
FontLibrary.Fonts.PlotTitle.FontSize = 16

FontLibrary.Fonts.DescriptionLeft.FontSize = 12
FontLibrary.Fonts.DescriptionLeft.FontWeight = 'bold'
FontLibrary.Fonts.DescriptionLeft.FontStyle = 'italic'

FontLibrary.Fonts.DescriptionBottom.FontSize = 10
FontLibrary.Fonts.DescriptionBottom.FontWeight = 'bold'
FontLibrary.Fonts.DescriptionBottom.FontStyle = 'italic'

FontLibrary.Fonts.DescriptionTop.FontSize = 10
FontLibrary.Fonts.DescriptionTop.FontWeight = 'bold'
FontLibrary.Fonts.DescriptionTop.FontStyle = 'italic'

FontLibrary.Fonts.ColumnDescriptionTop.FontSize = 9
FontLibrary.Fonts.ColumnDescriptionBottom.FontSize = 9
FontLibrary.Fonts.RowDescriptionLeft.FontSize = 9
FontLibrary.Fonts.RowDescriptionRight.FontSize = 9
FontLibrary.Fonts.AxisX.FontSize = 8
FontLibrary.Fonts.AxisY.FontSize = 8
FigureWidthHeight = (6.69291, 10.1181-1.0)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Используемый девайс:', torch.cuda.get_device_name(device) if torch.cuda.is_available() else 'ЦП')

Используемый девайс: NVIDIA A100-SXM4-80GB


# Построение изображения с помощью линзы

## Предопределение функций

### Функция генерации результатов построение изображения в линзе

In [8]:
def abstract_lens_image_construction(
    full_model:CompositeModel,
    propagation_module_a:AbstractPropagator,
    propagation_module_b:AbstractPropagator,
    focus:float,
    scales:Iterable[int],
    images:torch.Tensor,
    device:torch.device = None
):  
    if hasattr(scales, '__len__'):
        scales_count:int = len(scales)
    else:
        scales_count = 0
        for scale in scales:
            scales_count += 1

    if len(images.shape) != 4: raise AttributeError
    
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    full_model.to(device)
    images = images.to(device)

    with torch.no_grad():
        results = []
        for scale_i, scale in enumerate(scales):
            a = focus*(1.0 + 1.0/scale)
            b = focus*(1.0 + scale)
            propagation_module_a.distance = a
            propagation_module_b.distance = b
    
            result = full_model.forward(images).cpu()
            results.append(result)

    return results
        


### Функция отрисовки результатов